In [4]:
import json
from authorization import *

In [5]:
with open('client-secrets-rnr.json','r+') as secrets_file:
    secrets = json.load(secrets_file)

client_id = secrets["client_id"]
client_secret = secrets["client_secret"]
redirect_url = secrets["redirect_url"]

In [8]:
access_token = get_access_token('access_token.json', client_id,
    client_secret,
    redirect_url, 'auth_code.txt'
    )

In [ ]:
output = get_auth_tokens()
if 'error' in output.keys():
    print("Error! ", output["error_description"])
else:
    with open('auth_tokens.json','w+') as file:
        json.dump(output, file, indent=4)

In [21]:
def get_auth_tokens(client_id, client_secret, redirect_url, auth_code):
    auth_string = client_id + ":" + client_secret
    auth_bytes = auth_string.encode("utf-8")
    auth_encoded = str(base64.b64encode(auth_bytes), "utf-8")
    
    token_endpoint = "https://accounts.spotify.com/api/token"
    data = {
        "grant_type":"authorization_code",
        "code":auth_code,
        "redirect_uri":redirect_url,
    }
    headers = {
        "Authorization": "Basic " + auth_encoded,
        "Content-type": "application/x-www-form-urlencoded"
    }
    result = requests.post(token_endpoint, headers=headers, data=data)
    json_results = json.loads(result.content)
    return json_results

In [22]:
get_auth_tokens(client_id, client_secret, redirect_url, auth_code)

{'access_token': 'BQD6rx49nL5HQ_bxvolD0wZwuUUBrVGQ10_XF0M22aZAlwMhYzeoncy4qtMSadMulFyAOP7M6JIVuOg25cKtO33zW7_o4LLoImkpyNZpz2Bd3PxDwTXtXEMKNgF863Yd1kcinMnuTVK89v8CRiEYhTOm_UHW724L148sEBuuDJ0ZxiNgh_7o4hUlSKSNPFYtlpglVKCjk1__BP48efHGUkYZFzsV0bzRMzDnuTot_aoKqDs',
 'token_type': 'Bearer',
 'expires_in': 3600,
 'refresh_token': 'AQCD-i_ghFN4IaDwzrjLRM5-InJKX96MVzP7v01p0tZx3WN14msoYo1dijpBMAi6JHNSKM3uzP_vZZ8a9SF0YyIlVoXzxnuvLePni7tFOdSGny3IZRXq5y5Rk97EsbUiNek',
 'scope': 'playlist-read-private playlist-read-collaborative user-library-read user-read-playback-state user-read-currently-playing user-read-recently-played user-read-email user-read-playback-position user-top-read user-read-private'}

In [5]:
with open('auth_tokens.json') as file:
    json_string = file.read()
    
print(json_string)

{
    "access_token": "BQAo5FEAAMhX565altgNUnVizKqRvcQLIOfR-c-MwDfsHwg_DdVBZasFRsgBqu0r5Byjnt-MImLH0agSI2J94bzfrlbfSO0543Zg0YABup49Q9ZKDu3RscmtElp13l3HfWQ4yont1S7sETPww4Ws5H6Fr4mFaVU3oykSbdSgT_MAZqsceAy63IEa7tIpRxLX5pw-i-5olJGNFkDNMyWFCQbEcOgcSNk-UoMDdtpCc3oo6IY",
    "token_type": "Bearer",
    "expires_in": 3600,
    "scope": "playlist-read-private playlist-read-collaborative user-library-read user-read-playback-state user-read-currently-playing user-read-email user-read-recently-played user-read-playback-position user-read-private user-top-read"
}


In [5]:
with open('access_token.json','r+') as file:
    auth_keys = json.load(file)
    
if 'refresh_token' not in auth_keys.keys():
    with open('refresh_token.json','r+') as file:
        refresh_keys = json.load(file)
    refresh_token = refresh_keys['refresh_token']
else:
    refresh_token = auth_keys['refresh_token']
tokens = refresh_access_tokens(client_id, client_secret, refresh_token)

In [6]:
tokens

{'access_token': 'BQBEYRTFhy5reX2I74TuvCxzquvFGO5lZy2Wm0DAgqkGY8ONEwMrc9OMlAGrCBsyiXzlX53U-2WniH5Ice82rOy_Zwt9mzKudRt9OEclYEKQyKbI9bE9F7Tk3a7AGFFqnmK9aNjp9s99pS8sSoxo3P-9Yu2gzMay6EUut8DBxg3zCip8UTJ21yZykP6hZMlyuYM86SeFhq-lRJQVBoF3c5P4mgvZYQEoCTFrkBq7LoXVdMg',
 'token_type': 'Bearer',
 'expires_in': 3600,
 'scope': 'playlist-read-private playlist-read-collaborative user-library-read user-read-playback-state user-read-currently-playing user-read-email user-read-recently-played user-read-playback-position user-read-private user-top-read'}

In [7]:
if 'refresh_token' not in tokens.keys():
    with open('refresh_token.json','w+') as file:
        file.write(json.dumps({"refresh_token":f"{refresh_token}"}))

with open('auth_tokens.json','w+') as file:
        json.dump(tokens, file, indent=4)

# Get User Details

In [8]:
with open('auth_tokens.json','r+') as file:
    auth_keys = json.load(file)
    
token = auth_keys["access_token"]

result = requests.get("https://api.spotify.com/v1/me", headers={"Authorization": "Bearer " + token,
        "Content-type": "application/x-www-form-urlencoded"})
json_results = json.loads(result.content)

print(json_results)

{'display_name': 'sdspot2034', 'external_urls': {'spotify': 'https://open.spotify.com/user/sdspot2034'}, 'href': 'https://api.spotify.com/v1/users/sdspot2034', 'id': 'sdspot2034', 'images': [{'url': 'https://i.scdn.co/image/ab67757000003b8280153af8b79b3144b24f1291', 'height': 64, 'width': 64}, {'url': 'https://i.scdn.co/image/ab6775700000ee8580153af8b79b3144b24f1291', 'height': 300, 'width': 300}], 'type': 'user', 'uri': 'spotify:user:sdspot2034', 'followers': {'href': None, 'total': 9}, 'country': 'IN', 'product': 'premium', 'explicit_content': {'filter_enabled': False, 'filter_locked': False}, 'email': 'shreyandas2034@gmail.com'}


In [ ]:
import datetime
yesterday = datetime.datetime.now() - datetime.timedelta(days=1)
# print(yesterday.timestamp()*1e3)
cdc_time = int(yesterday.timestamp()*1e3)

# Get All Songs Played in the last one day

In [ ]:
with open('auth_tokens.json','r+') as file:
    auth_keys = json.load(file)
    
token = auth_keys["access_token"]

result = requests.get("https://api.spotify.com/v1/me/player/recently-played", 
        headers={"Authorization": "Bearer " + token}, params={"after": str(cdc_time), "limit":"50"})


In [ ]:
json_results = json.loads(result.content)
print(json_results)

In [ ]:
import pprint
pprint.pprint(json_results)

## EDA over recent history

### Number of songs played

In [ ]:
len(json_results['items'])

In [ ]:
type(json_results['items'])

### Fetch details of a song

#### See available data for a song

In [ ]:
json_results['items'][0]

In [ ]:
json_results['items'][0].keys()

In [ ]:
json_results['items'][0]['track']

In [ ]:
list(json_results['items'][0]['track'].keys())

In [ ]:
track_details = {k: v for k, v in json_results['items'][0]['track'].items() if k not in ('available_markets')}

if 'album' in track_details:
    del track_details['album']['images']
    del track_details['album']['available_markets']

track_details
